# Gemma 3 9B Fine-tuning - 고전문학 사고유도 AI

**완벽한 학습 노트북 - A100 GPU 최적화 🚀**

## Step 1: 패키지 설치 (필수!)

In [ ]:
# 기존 패키지 제거
!pip uninstall -y torch torchvision torchaudio transformers peft trl bitsandbytes numpy scipy

# 호환되는 numpy, scipy 설치
!pip install numpy==1.26.4 scipy==1.13.1

# PyTorch 2.4.0 설치 (CUDA 12.1)
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

# 학습 라이브러리 설치
!pip install transformers==4.44.2 peft==0.12.0 trl==0.9.6 bitsandbytes==0.43.3 datasets==2.21.0 accelerate==0.33.0

print("\n" + "="*60)
print("✅ 설치 완료!")
print("="*60)
print("\n⚠️ 중요: 지금 Kernel을 재시작하세요!")
print("   Kernel → Restart Kernel")
print("\n재시작 후 다음 셀(Cell 3)부터 실행하세요.\n")

## Step 2: 환경 확인

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer
import os

print("="*60)
print("환경 확인")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
print("="*60)

## Step 3: GPU 상세 확인

In [ ]:
print("="*60)
print("GPU 상태 확인")
print("="*60)

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\n🔥 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   총 메모리: {props.total_memory / 1024**3:.2f} GB")
        print(f"   Compute Capability: {props.major}.{props.minor}")
        
    print(f"\n💾 현재 메모리 사용:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
else:
    print("⚠️ GPU를 찾을 수 없습니다!")
    
print("\n" + "="*60)

## Step 4: 데이터 다운로드

In [ ]:
# Cloud Storage에서 균형잡힌 데이터셋 다운로드
!gsutil cp gs://knu-team-03-data/classical-literature/gemma/train_balanced.jsonl ./
!gsutil cp gs://knu-team-03-data/classical-literature/gemma/valid_balanced.jsonl ./

print("\n" + "="*60)
print("✅ 균형잡힌 데이터셋 다운로드 완료!")
print("   - 고전문학 50% + AI Hub 우수 샘플 50%")
print("   - 총 822개 샘플 (train: 657, valid: 165)")
print("="*60)

## Step 5: Hugging Face 로그인

In [ ]:
from huggingface_hub import login

# Hugging Face 토큰 (자동 설정됨)
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

login(token=HF_TOKEN)
print("✅ Hugging Face 로그인 완료!")
print("✅ Gemma 모델 접근 권한 확인됨")

## Step 6: 데이터셋 로드

In [ ]:
# 균형잡힌 데이터셋 로드
train_dataset = load_dataset('json', data_files='train_balanced.jsonl', split='train')
valid_dataset = load_dataset('json', data_files='train_balanced.jsonl', split='train')

print("="*60)
print("✅ 데이터셋 로드 완료!")
print("="*60)
print(f"Train: {len(train_dataset)}개 (고전 50% + AI Hub 우수 50%)")
print(f"Valid: {len(valid_dataset)}개")
print(f"\n⚡ 예상 학습 시간: 1.5-2시간 (A100 40GB GPU)")
print("="*60)
print(f"\nSample:\n{train_dataset[0]}")

## Step 7: 모델 및 토크나이저 로드

In [ ]:
model_name = "google/gemma-2-9b-it"

# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("모델 로드 중... (3-5분 소요)")

# 모델 로드 (Gemma2 최적화: eager attention)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"  # Gemma2 권장 설정
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("\n✅ 모델 로드 완료 (eager attention)")

## Step 8: LoRA 설정

In [ ]:
# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# 모델 준비
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

print("="*60)
print("LoRA 설정 완료")
print("="*60)
model.print_trainable_parameters()
print("="*60)

## Step 9: 데이터 포맷팅

In [ ]:
def formatting_func(examples):
    """데이터를 Gemma 형식으로 변환"""
    texts = []
    for instruction, input_text, output in zip(examples['instruction'], examples['input'], examples['output']):
        text = f"""<start_of_turn>user
{instruction}

{input_text}<end_of_turn>
<start_of_turn>model
{output}<end_of_turn>"""
        texts.append(text)
    return texts

# 테스트
print("="*60)
print("데이터 포맷팅 테스트")
print("="*60)
print(formatting_func(train_dataset[:1])[0][:500])
print("...")
print("="*60)

## Step 10: Trainer 설정 (A100 40GB 최적화 🚀)

In [ ]:
output_dir = "./gemma3-classical-lit-finetuned"

# A100 40GB 최적화 설정
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    
    # ⚡ A100 40GB - 배치 크기 대폭 증가!
    per_device_train_batch_size=4,  # T4: 1 → A100: 4
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # T4: 16 → A100: 4
    
    # 학습률 및 스케줄러
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    
    # 로깅 및 평가
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    
    # 체크포인트 저장
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,
    
    # 정밀도 최적화 (A100은 tf32 지원!)
    bf16=True,
    tf32=True,  # ⚡ A100만 가능!
    
    # 메모리 최적화
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    max_grad_norm=1.0,
    
    # 데이터 로딩 최적화
    dataloader_num_workers=4,  # T4: 2 → A100: 4
    dataloader_pin_memory=True,
    
    # 기타
    report_to="none",
    load_best_model_at_end=False,
)

# SFTTrainer 생성
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    formatting_func=formatting_func,
    packing=False,
    max_seq_length=1024,  # A100은 메모리 여유 → 1024
    dataset_text_field=None,
    tokenizer=tokenizer,
)

print("="*60)
print("✅ Trainer 설정 완료 (A100 40GB 최적화)")
print("="*60)
print(f"🚀 Effective batch size: 16 (per_device=4 × grad_accum=4)")
print(f"🚀 Max sequence length: 1024 (T4 대비 2배!)")
print(f"⚡ 예상 학습 시간: 1.5-2시간 (T4 대비 5배 빠름!)")
print("="*60)

## Step 11: 학습 시작!

In [ ]:
print("="*60)
print("🚀 학습 시작...")
print("="*60)

trainer.train()

print("\n" + "="*60)
print("✅ 학습 완료!")
print("="*60)

## Step 12: 모델 저장

In [ ]:
# 로컬 저장
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ 모델 저장 완료: {output_dir}")

# Cloud Storage에 업로드
!gsutil -m cp -r {output_dir} gs://knu-team-03-data/classical-literature/models/
print("\n✅ Cloud Storage 업로드 완료")
print("   gs://knu-team-03-data/classical-literature/models/gemma3-classical-lit-finetuned")

## Step 13: 테스트

In [ ]:
# 추론 테스트
test_input = """<start_of_turn>user
다음 지문을 읽고 질문에 답하세요. 학생의 사고를 유도하며 답변을 작성하세요.

[지문]
춘향전의 한 장면...

[질문]
이몽룡이 신분을 숨긴 이유는 무엇인가요?<end_of_turn>
<start_of_turn>model
"""

inputs = tokenizer(test_input, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)
response = tokenizer.decode(outputs[0], skip_special_tokens=False)

print("="*60)
print("모델 응답")
print("="*60)
print(response)
print("="*60)